# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Подгружаем необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import scipy.stats as st
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

## Обучение и проверка модели

Обрабатываем данные. Удаляем лишний столбец с id скважины, который не несет в себе полезной для обучения моделей информации.

In [2]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
print(data_1.info())
display(data_1.head())
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
print(data_2.info())
display(data_2.head())
print(data_2.info())
data_3 = pd.read_csv('/datasets/geo_data_2.csv')
display(data_3.head())

data_1 = data_1.drop('id', axis=1)
data_2 = data_2.drop('id', axis=1)
data_3 = data_3.drop('id', axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Разделяем данные на тренировочные и валидационные, нормализуем данные, затем обучаем модель линейной регресии по 1 региону. Рассчитываем ее точность и предсказание среднего объема в одной скважине. Проделываем то же самое и для других регионов.

In [3]:
features_1 = data_1.drop('product', axis=1)
target_1 = data_1['product']
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, test_size=0.25, random_state=12345)
numeric = ['f0', 'f1', 'f2']
scaler = StandardScaler()
scaler.fit(features_train_1[numeric])
features_train_1[numeric] = scaler.transform(features_train_1[numeric])
features_valid_1[numeric] = scaler.transform(features_valid_1[numeric])

model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_1 = pd.Series(model_1.predict(features_valid_1))
rmse_1 = (mean_squared_error(predictions_1, target_valid_1))**(0.5)
average_1 = sum(predictions_1) / len(predictions_1)
    
print('RMSE =', rmse_1)
print('average =', average_1)

RMSE = 37.5794217150813
average = 92.59256778438008


1 регион. Средний прогнозируемый запас в скважине - около 92,6. При этом корень из средней квадратичной ошибки - около 37.58, то есть возможная погрешность довольно высока.

In [4]:
features_2 = data_2.drop('product', axis=1)
target_2 = data_2['product']
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, test_size=0.25, random_state=12345)
numeric = ['f0', 'f1', 'f2']
scaler = StandardScaler()
scaler.fit(features_train_2[numeric])
features_train_2[numeric] = scaler.transform(features_train_2[numeric])
features_valid_2[numeric] = scaler.transform(features_valid_2[numeric])

model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predictions_2 = pd.Series(model_2.predict(features_valid_2))
rmse_2 = (mean_squared_error(predictions_2, target_valid_2))**(0.5)
average_2 = sum(predictions_2) / len(predictions_2)
    
print('RMSE =', rmse_2)
print('average =', average_2)

RMSE = 0.893099286775617
average = 68.7285468954458


2 регион. Средний прогнозируемый запас в скважине - около 68,72, что значительно меньше, чем в 1. При этом корень из средней квадратичной ошибки - около 0.9, что является отличным результом. очень высокая точность.

In [5]:
features_3 = data_3.drop('product', axis=1)
target_3 = data_3['product']
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(features_3, target_3, test_size=0.25, random_state=12345)
numeric = ['f0', 'f1', 'f2']
scaler = StandardScaler()
scaler.fit(features_train_3[numeric])
features_train_3[numeric] = scaler.transform(features_train_3[numeric])
features_valid_3[numeric] = scaler.transform(features_valid_3[numeric])

model_3 = LinearRegression()
model_3.fit(features_train_3, target_train_3)
predictions_3 = pd.Series(model_3.predict(features_valid_3))
rmse_3 = (mean_squared_error(predictions_3, target_valid_3))**(0.5)
average_3 = sum(predictions_3) / len(predictions_3)
    
print('RMSE =', rmse_3)
print('average =', average_3)

RMSE = 40.02970873393434
average = 94.96504596800506


1 регион. Средний прогнозируемый запас в скважине - около 94,97. При этом корень из средней квадратичной ошибки - около 40.03, то есть здесь самая высокая погрешность и самый высокий предсказанный запас.

## Подготовка к расчёту прибыли

Найдем точку безубыточности и необходимый для безубыточности запас нефти в целом и в каждой скважине, если их 200.

In [6]:
num_of_holes = 200
budget = 10000000000
income_1_bar = 450000
min_salary = budget/income_1_bar
print('Для безубыточной разработки требуется баррелей нефти:', min_salary)
print('В среднем с одной скважины нужно получить баррелей нефти:', min_salary/num_of_holes)

Для безубыточной разработки требуется баррелей нефти: 22222.222222222223
В среднем с одной скважины нужно получить баррелей нефти: 111.11111111111111


Необходимый запас нефти в каждой скважине в среднем - 111,11, что выше, чем среднее предсказанное по скважинам число в каждом из регионов. Соответственно, для того, чтобы добыча была рентабельной, очень важно выбрать именно скважины с самым большим запасом. Случайный выбор, с большой долей вероятности, приведет к убыткам.

## Расчёт прибыли и рисков 

Создадим функция для рассчета максимальной прибыли для каждого из регионов. Для этого предскажем 200 лучших скважин, по их индексам возьмем целевые значения по емкости скважин и просуммируем. Затем вычтем себестоимость.

In [7]:
def profit(predictions, target):
    target.reset_index(drop=True, inplace=True)
    predictions.reset_index(drop=True, inplace=True)
    top_preds = predictions.sort_values(ascending=False)
    top_target = target[top_preds.index][:200]
    revenue = top_target.sum() * income_1_bar
    return revenue - budget
print("Прибыль первого региона", profit(predictions_1, target_valid_1))
print("Прибыль второго региона", profit(predictions_2, target_valid_2))
print("Прибыль третьего региона", profit(predictions_3, target_valid_3))

Прибыль первого региона 3320826043.1398506
Прибыль второго региона 2415086696.681511
Прибыль третьего региона 2710349963.5998325


Видно, что модель предсказывает самую высокую прибыль в 1 регионе, затем в третьем, и самую низкую во втором. При условии выбора из всех скажин 200 самых объемных.

Создадим функцию для расчета средней прибыли, доверительного 95% интервала и риска уйти в убыток. Воспользуемся техникой Bootstrap.

In [8]:
state = np.random.RandomState(12345)

def conf_interval_and_risk(predictions, target):
    revenue = []
    target.reset_index(drop=True, inplace=True)
    for _ in range(1000):
        target_sample = target.sample(n = 500, replace=True, random_state=state)
        predictions_sample = predictions[target_sample.index]
        revenue.append(profit(predictions_sample, target_sample))   
      
       
    revenue = pd.Series(revenue)
    lower = revenue.quantile(0.025)
    higher = revenue.quantile(0.975)
    mean_revenue = int(sum(revenue) / len(revenue))
    count = 0
    for a in revenue:
        if a < 0:
            count += 1
    risk = count/len(revenue) * 100

    return (lower, higher, mean_revenue, risk)

print(conf_interval_and_risk(predictions_1, target_valid_1))
print(conf_interval_and_risk(predictions_2, target_valid_2))
print(conf_interval_and_risk(predictions_3, target_valid_3))

(-111215545.89049526, 909766941.5534226, 396164984, 6.9)
(78050810.7517417, 862952060.2637234, 461155817, 0.7000000000000001)
(-112227625.37857565, 934562914.5511636, 392950475, 6.5)


По получившимся данным, единственным регионом с риском получить убытки ниже 2,5% оказался 2 регион с риском, равным 0.7%. При этом средняя предсказанная прибыль равна примерно 461 миллиону рублей, что является наибольшим значением для 3 регионов.. Кроме того, в доверительном 95-% интервале даже минимальное значение прибыли положительное, и это единственный такой регион из всех. Исходя из этих данных, очевидно, что наиболее перспективным для разработки является 2 регион.

<div style="background: #cceeaa; padding: 5px; border: 1px solid green; border-radius: 5px;">

Очень рекомендую посмотреть видео от Глеба Михайлова!)<br>
**[Линейная Регрессия для Дата Саентиста](https://www.youtube.com/watch?v=QZJ94igWVxQ&t=19s)**     <br> 
    
**[Вот здесь есть отличное объяснение с примером по мультиколлинеарности.](https://habr.com/ru/company/akbarsdigital/blog/592493/)** <br>
    
**[И здесь интересная статься - коротко о мультиколлинеарности и что с этим делать.](https://medium.com/analytics-vidhya/removing-multi-collinearity-for-linear-and-logistic-regression-f1fa744f3666)** <br>   
   
**[Для лучшего понимания статистических методов очень рекомендую посмотреть короткие видео-лекции Карпова на степике](https://stepik.org/lesson/8095/step/1?unit=1371)**    <br>
    


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован